### <b> Apple Classifier </b> 

This notebook is a Deep Learning project  to train and deploy an image classifier using FastAI's API.

In [ ]:
from fastbook import *
from fastai.vision.widgets import *

##### 1. Gathering Data from the web

In [ ]:
apple_types = ['red', 'green', 'logo']
path = Path('apple')

In [ ]:
for a in apple_types:
    dest = (path/a)
    results = search_images_ddg(f'{a} apple images')
    for i in range(len(results)):
        try:
            download_url(results[i], dest / f'{i}.jpg')
        except Exception as e:
            print(f"Error downloading {results[i]}: {e}")